In [1]:
!pip install git+https://github.com/huggingface/diffusers transformers accelerate


  Cloning https://github.com/huggingface/diffusers to /tmp/pip-req-build-yj53tg9s
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers /tmp/pip-req-build-yj53tg9s
  Resolved https://github.com/huggingface/diffusers to commit 534f5d54faf24a72a0e2bff2f6e6379ea519c4ed
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 6.6 MB/s eta 0:00:00
  Created wheel for diffusers: filename=diffusers-0.27.0.dev0-py3-none-any.whl size=1980366 sha256=5303f17f863eda958f6b1ec5c00b649ea3867fdbd15b6be8cb2c9416ef5e79d5
  Stored in directory: /tmp/pip-ephem-wheel-cache-8dx9qzee/wheels/f7/7d/99/d361489e5762e3464b3811bc629e94cf5bf5ef44dd5c3c4d52
Successfully built diffusers


In [12]:
import torch
from diffusers import DiffusionPipeline, DPMSolverMultistepScheduler
from diffusers.utils import export_to_video

# load pipeline
pipe = DiffusionPipeline.from_pretrained("damo-vilab/text-to-video-ms-1.7b", torch_dtype=torch.float16, variant="fp16")
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

# optimize for GPU memory
pipe.enable_model_cpu_offload()
pipe.enable_vae_slicing()

# generate
prompt = "HorseRide in Beach"
video_frames = pipe(prompt, num_inference_steps=25, num_frames=200).frames


Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
video_path = export_to_video(video_frames)

In [14]:
video_path = export_to_video(video_frames[0], output_video_path="HorseRide.mp4", fps=30)

In [17]:
video_path = export_to_video(video_frames[0], output_video_path="HorseRide2.mp4", fps=24)

In [18]:
!pip install moviepy
!apt-get install imagemagick

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript gsfonts imagemagick-6-common
  imagemagick-6.q16 libdjvulibre-text libdjvulibre21 libfftw3-double3 libgs9 libgs9-common libidn12
  libijs-0.35 libjbig2dec0 libjxr-tools libjxr0 liblqr-1-0 libmagickcore-6.q16-6
  libmagickcore-6.q16-6-extra libmagickwand-6.q16-6 libnetpbm10 libwmflite-0.2-7 netpbm
  poppler-data
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre ghostscript-x imagemagick-doc
  autotrace cups-bsd | lpr | lprng enscript gimp gnuplot grads hp2xx html2ps libwmf-bin mplayer
  povray radiance sane-utils texlive-base-bin transfig ufraw-batch libfftw3-bin libfftw3-dev
  inkscape poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho fonts-japanese-gothic
  | fonts-ipafont-gothic fonts-arphic-ukai fonts-arphic-uming fonts-nan

In [ ]:
from moviepy.editor import VideoFileClip

def cut_video(input_path, start_time, end_time, output_folder):
    # Load the video clip
    video_clip = VideoFileClip(input_path)
    # Cut the video clip
    cut_video_clip = video_clip.subclip(start_time, end_time)
    # Specify the output file path for the cut video
    output_file_path = f"{output_folder}/output_cut_video.mp4"
    # Save the cut video to the specified output file
    cut_video_clip.write_videofile(output_file_path, codec="libx264", audio_codec="aac")
    # Close the video clips
    video_clip.close()
    cut_video_clip.close()


# Get user input for video path
input_path = input("Enter the input video path: ")
# Get user input for start and end times (in seconds)
start_time = float(input("Enter the start time (in seconds): "))
end_time = float(input("Enter the end time (in seconds): "))
# Get user input for the output folder
output_folder = input("Enter the output folder path: ")
# Call the cut_video function
cut_video(input_path, start_time, end_time, output_folder)


In [19]:
from moviepy.editor import VideoFileClip, concatenate_videoclips

def concatenate_videos(video_paths, output_path, output_extension):
    # Load the video clips
    video_clips = [VideoFileClip(path) for path in video_paths]

    # Concatenate the video clips
    concatenated_clip = concatenate_videoclips(video_clips, method="compose")

    # Write the concatenated video to the specified output file
    final_output_path = f"{output_path}.{output_extension}"
    concatenated_clip.write_videofile(final_output_path, codec="libx264", audio_codec="aac")

    # Close the video clips
    for clip in video_clips:
        clip.close()
    concatenated_clip.close()

# Get user input for video paths and output file path
video_path1 = input("Enter the first video path: ")
video_path2 = input("Enter the second video path: ")
output_path = input("Enter the desired output path (directory): ")
output_extension = input("Enter the desired output file extension (e.g., mp4, avi): ")

# Call the concatenate_videos function
concatenate_videos([video_path1, video_path2], output_path, output_extension)


Enter the first video path: /content/ducksinlakes.mp4
Enter the second video path: /content/HorseRide.mp4
Enter the desired output path (directory): /content/utput
Enter the desired output file extension (e.g., mp4, avi): mp4
Moviepy - Building video /content/utput.mp4.
Moviepy - Writing video /content/utput.mp4



t:  49%|████▉     | 197/401 [00:02<00:03, 61.61it/s, now=None]WARNING:py.warnings:/usr/local/lib/python3.10/dist-packages/moviepy/video/io/ffmpeg_reader.py:123: UserWarning: Warning: in file /content/ducksinlakes.mp4, 196608 bytes wanted but 0 bytes read,at frame 200/201, at time 6.67/6.67 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready /content/utput.mp4
